# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Project 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: **
Classifição trata-se de atribuir um rótulo a determinada amostra, dadas as entradas o sistema deve analiza-las e como saída dar uma resposta discreta, não necessariamente binária, porém na maioria das vezes binária, um "sim ou não", "macho ou fêmea", coisas do tipo. Tal nome deve-se ao fato que o sistema vai realmente classificar a amostra entre rótulos pré-definidos. Já a Regressão trata-se de uma resposta contínua. Dado um conjunto de amostras, o sistema encontrará uma função que melhor descreva esses dados. Não se trata de rótulos pré-definidos, a saída será numérica e contínua.
Portanto, o problema apresentado trata-se de uma **classificação**, pois cada amostra deve ser classificado com um "sim" ou "não".

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [126]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [127]:
# TODO: Calcule o número de estudante
n_students = len(student_data)

# TODO: Calcule o número de atributos
n_features = len(student_data.columns)-1 #número de colunas em student_data menos a coluna alvo 'passed'

# TODO: Calcule o número de alunos aprovados
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calcule o número de alunos reprovados
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calcule a taxa de graduação
grad_rate = (float(n_passed)/float(n_students))*100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [128]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo, 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (em inglês: _dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [129]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [130]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.cross_validation import train_test_split

# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test, random_state = 82)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **
1. Árvore de Decisão. 
    - Aplicação: Reconhecimento de objeto - Árvore de decisão foi utilizada para reconhecimento de objeto tridimensional em "LILLY SPIRKOVSKA. Three dimensional object recognition using similar triangles and decision trees. Pattern Recognition, 26(5):727, May 1993" e em visão de alto nível em "Y. KODRATOFF AND S. MOSCATELLI. Machine learning for object recognition and scene analysis. Internationa Journal of Pattern recognition and AI, 8(1):259--304, 1994".
    - Vantagem: Uma grande vantagem do modelo de árvore de decisão é sua natureza transparente. Ao contrário de outros modelos de tomada de decisão, a árvore de decisão torna explícitas todas as alternativas possíveis e traça cada alternativa para sua conclusão em uma única visão, permitindo uma comparação fácil entre as várias alternativas. Também é interessante pela facilidade no uso, a árvore de decisão fornece uma ilustração gráfica do problema e várias alternativas em um formato simples e fácil de entender que não requer muita explicação. Fonte: http://www.brighthubpm.com/project-planning/106000-advantages-of-decision-tree-analysis/
    - Desvantagem: Uma desvantagem da árvore de decisão é que ela tende a sobreajustar, especialmente se os dados tiverem uma grande quantidade de características, pois a arvore pode crescer muito.
    - Creio que o modelo será bom para o problema pois tratam-se de características binárias, o sim ou não em cada característica já remete a árvore de decisão, assim como foi feito, só que de forma manual, no Projeto Exploratório.

2. SVM - Support Vector Machine
    - Aplicação: Foi utilizado SVM para a previsão de adesão à medicação em pacientes com insuficiência cardíaca. Fonte: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3092139/
    - Vantagens: Eficaz em espaços dimensionais elevados; Se mantém eficaz nos casos em que o número de dimensões é maior que o número de amostras; Usa um subconjunto de pontos de treinamento na função de decisão (chamado de vetores de suporte), por isso também é eficiente em memória; Versátil: diferentes funções do Kernel podem ser especificadas para a função de decisão. Fonte: http://scikit-learn.org/stable/modules/svm.html
    - Desvantagens: Se o número de características for muito maior do que o número de amostras, é necessário um ajuste nas funções do Kernel  para evitar o sobreajuste.  Fonte: http://scikit-learn.org/stable/modules/svm.html
    - Porque usar SVM: Como o conjunto de dados possuem muitas características, creio que SVM poderá ter um bom desempenho, além da enconomia de memória que bate com o baixo custo solicitado no projeto.
    
3. Gradient Boosting
    - Aplicação: Gradient Boosting foi utilizado como um modelo de regreção para mapear sinais eletromiográficos para controlar uma mão robótica em Vogel et al.(2011). Fonte: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3885826/
    - Vantagens: Dados heterogêneos (características medidas em escala diferente); Suporta diferentes funções de perda (e.g. huber); Detecta automaticamente interações de recursos (não-lineares) Fonte: https://orbi.ulg.ac.be/bitstream/2268/163521/1/slides.pdf
    - Desvantagens: Requer ajuste cuidadoso; Treinamento lento (porém predição rápida); Não pode ser estrapolado. Fonte: https://orbi.ulg.ac.be/bitstream/2268/163521/1/slides.pdf
    - Porque usar: Gradient Boosting consegue capturar relações complexas nos dados; existe a possibilidade de experimentar diversos valores nos hiperparâmetros para encontrar um melhor ajuste; como vou testar a Árvore de Decisão, quero comparar o desempenho de uma única árvore com um método por agregação que usa árvores.

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [131]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [136]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn import tree
from sklearn import svm
from sklearn import ensemble

# TODO: Inicialize os três modelos
my_random_state=82
clf_A = tree.DecisionTreeClassifier(random_state = my_random_state)
clf_B = svm.SVC(random_state = my_random_state)
clf_C = ensemble.GradientBoostingClassifier(random_state = my_random_state)


# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
# train_predict(clf, X_train, y_train, X_test, y_test)

for clf in [clf_A, clf_B, clf_C]:
    for n_train in [100, 200, 300]:
        train_predict(clf, 
                      X_train[:n_train], y_train[:n_train], 
                      X_test, y_test)

Treinando um DecisionTreeClassifier com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0010 segundos
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de treino: 1.0000.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.5965.
Treinando um DecisionTreeClassifier com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0010 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 1.0000.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.6557.
Treinando um DecisionTreeClassifier com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 1.0000.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.7313.
Treinando um SVC com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsõ

### Resultados Tabulados
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - DecisionTreeClassifier**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |       0.0010         |         0.0010              |          1.0               |         0.5965        |
| 200                                |       0.0020         |         0.0010              |          1.0               |         0.6557        |
| 300                                |       0.0020         |         0.0000              |          1.0               |        0.7313        |

** Classificador 2 - SVC**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |       0.0020         |         0.0020              |         0.8630             |         0.8267        |
| 200                                |       0.0050         |         0.0020              |         0.8857             |         0.8514        |
| 300                                |       0.0090         |         0.0030              |         0.8547             |        0.8462       |

** Classificador 3 - GradientBoostingClassifier**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |       0.0970         |         0.0000              |           1.0              |         0.7559        |
| 200                                |       0.0660         |         0.0000              |           1.0              |         0.7846        |
| 300                                |       0.0870         |         0.0000              |           0.9706           |         0.8369        |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **
Apesar do modelo SVC ter atingido uma pontuação de 84.62% nos dados de testes, acredito que o modelo GradientBoosting conseguirá se sair melhor após a calibração, pois até agora o modelo está sobreajustado nos dados de treinamento, mas ao realizar uma suaviação das árvores o modelo final certamente terá melhor desempenho nos dados de teste. Além disso, depois da calibração, o modelo final gastará pouco recurso de memória e terá um desempenho rápido, podendo ser utilizado com um bom desempenho em máquinas com baixo poder processamento e sem utilizar muita memória, diminuindo o custo de investimento.

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **
O modelo final, GradientBoosting, trata-se de um modelo construído a partir de várias árvores de decisão (o primeiro modelo que foi descartado era constituído por apenas uma árvore). A árvore de decisão verifica se cada característica do aluno o leva mais para próximo de uma aprovação ou reprovação, e após analisar várias características do aluno, ela decide se este aluno será aprovado ou reprovado.  O GradientBoosting será uma combinação de vários "aprendizes fracos". Um "aprendiz fraco" é definido como um modelo aprendiz que é melhor do que o acaso, portanto, se simplesmente sorteassemos os rótulos dos alunos ao acaso, digamos que teríamos 50% de chance de acerto, um "aprendiz fraco" seria um modelo qualquer que acertasse mais que os 50% do acaso. Definido agora o "aprendiz fraco" podemos dizer que o GradientBoosting começará com uma função F que será igual ao primeiro aprendiz fraco encontrado. Agora ele irá calcular outros aprendizes fracos que serão adicionados a função F com um peso. A cada ciclo, a máquina encontrará um aprendiz fraco que ao ser adicionado à função F tornará essa função um pouco melhor, ou seja, F conseguirá errar um pouco menos ao prever os rótulos dos alunos. Para isso vamos supor que exista um aprendiz perfeito que ao ser adicionado a função F, esta preveja com exatidão o resultado de todos os alunos. Ou seja, esse aprendiz perfeito seria a diferença entre a resposta final que procuramos e a fornecida pela função F atual. Ao invés de encontrar exatamente o aprendiz perfeito, o que não é pouco provável ou quase impossível, o GradientBoosting se utiliza de um aprendiz fraco que é a diferença entre os rótulos alvo e a função F atual com um valor residual. Este valor residual é um gradiente negativo, em relação a função F, do erro quadrático daquela diferença. Então para concluir, podemos dizer que a cada ciclo, a função F será incrementada do novo aprendiz fraco com um peso, este peso é o gradiente deste aprendiz, levando a próxima função F, que estará mais próxima da resposta final. Assim, o sistema rodará vários ciclos, e a cada ciclo estará melhor classificando os dados.

### Implementação: Calibrando o Modelo
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [137]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters = {'max_depth':range(1,4), 'max_features':range(10,20), 'learning_rate':[0.01, 0.05, 0.1]}

# TODO: Inicialize o classificador
clf = ensemble.GradientBoostingClassifier(random_state = my_random_state)

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score)

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(clf, parameters)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

parametros_otimos = clf.get_params()
print "\nOs parâmetros ótimos encontrados foram:"
print "max_depth = {}" .format(parametros_otimos['max_depth'])
print "max_features = {}" .format(parametros_otimos['max_features'])
print "learning_rate = {}" .format(parametros_otimos['learning_rate'])

As previsões foram feitas em 0.0000 segundos.
O modelo calibrado tem F1 de 0.8731 no conjunto de treinamento.
As previsões foram feitas em 0.0000 segundos.
O modelo calibrado tem F1 de 0.8571 no conjunto de teste.

Os parâmetros ótimos encontrados foram:
max_depth = 2
max_features = 15
learning_rate = 0.05


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **
    O modelo final obteve uma pontuação F1 nos dados de treinamento de 0.8731 e nos dados de teste de 0.8571. No modelo não calibrado a pontuação de treinamento havia sido de 0.9706 e de teste de 0.8369. Claramente se vê que o modelo não calibrado estava sobreajustado aos dados de treinamento, porque conseguia uma pontuação alta no treinamento mas a pontuação caia bastante quando se tratava do conjunto de teste. Na calibração as árvores foram podadas, passaram à profundidade máxima de 2 e um número máximo de características de 15. Ainda foi utilizada uma taxa de aprendizagem menor, 0.05 (antes 0.1). Tais mudanças de parâmetros suavizam o aprendizado de cada árvore separadamente, assim, na soma ponderada de todas as árvores (através do boosting) o classificador final tem um modelo também mais suave, não sobreajustado, errando mais nos dados de treinamento mas também acertando mais nos dados de teste.
    
Nota: Ainda foi encontrada uma parametrização em que o modelo acertava 87% dos dados de teste, porém apenas 82% dos dados de treinamento, essa parametrização foi descartada porque achei que era um caso especial em que o modelo, por acaso, estava sobreajustado nos dados de testes. Entendi que a parametrização dada como resposta final corresponde a um modelo mais generalizado,  que mesmo acertando menos nos dados de teste (85.71%) tem uma maior chance de ter um melhor desempenho para novos dados, pois é mais coerente acertar 87% no treinamento e 85% no teste do que 82% no treimamento e 87% no teste.

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.